In [1]:
import sys
sys.path.append('../../src')

from common import common, transform, json
from tracker import main as tr
from indivisual_activity import main as ia
from indivisual_activity.indivisual_activity import IndivisualActivity
from group_activity import main as ga
from display.display import display
import os
import cv2
from tqdm import tqdm

In [2]:
room_num = '09'
date = '20210304'
name = 'pass1'
method = 'passing'
THRESHOLD = 0.6

In [3]:
video_path = os.path.join(
        common.data_dir, '{0}/{1}/{2}/video/AlphaPose_{2}.mp4'.format(room_num, date, name))
out_dir = os.path.join(
    common.data_dir, '{0}/{1}/{2}/out/'.format(room_num, date, name))
field_path = os.path.join(
    common.data_dir, 'field.png')
keypoints_path = os.path.join(
    common.data_dir, '{0}/{1}/{2}/json/alphapose-results.json'.format(room_num, date, name))
tracking_json_path = os.path.join(
    common.data_dir, '{0}/{1}/{2}/json/tracking.json'.format(room_num, date, name))
indivisual_activity_json_path = os.path.join(
    common.data_dir, '{0}/{1}/{2}/json/indivisual_activity.json'.format(room_num, date, name))
group_activity_json_path = os.path.join(
    common.data_dir, '{0}/{1}/{2}/json/group_activity.json'.format(room_num, date, name))

In [4]:
# homography
field_raw = cv2.imread(field_path)
p_video = common.homo[room_num][0]
p_field = common.homo[room_num][1]
homo = transform.Homography(p_video, p_field, field_raw.shape)

In [5]:
# tr.main(keypoints_path, tracking_json_path)

In [6]:
# ia.main(tracking_json_path, indivisual_activity_json_path, homo)

In [7]:
# ga.main(
#     indivisual_activity_json_path,
#     group_activity_json_path,
#     homo,
#     field_raw,
#     method)

In [8]:
# display(
#     video_path,
#     out_dir,
#     indivisual_activity_json_path,
#     group_activity_json_path,
#     field_raw,
#     method)

In [5]:
indivisual_activity_datas = json.load(indivisual_activity_json_path)

In [ ]:
for data in indivisual_activity_datas:
    